In [1]:
import pandas as pd
from sodapy import Socrata

In [2]:
client = Socrata("data.cdc.gov", None)
results = client.get("hk9y-quqm", limit=350000)

# Convert to pandas DataFrame
doom_df = pd.DataFrame.from_records(results)
doom_df = doom_df.drop(columns=["data_as_of", "start_date", "end_date", "group", "icd10_codes", "flag", "number_of_mentions", "year", "month"])
doom_df = doom_df.loc[(doom_df["condition_group"] == "COVID-19")]
doom_df = doom_df[doom_df["state"] != "United States"]
doom_df = doom_df.loc[(doom_df["age_group"] == "All Ages")]
doom_df = doom_df.reset_index()
doom_df = doom_df.head(53)
doom_df = doom_df.drop(columns=["index", "condition_group", "condition", "age_group"])
doom_df["covid_19_deaths"] = doom_df["covid_19_deaths"].astype(int)

In [3]:
client = Socrata("data.cdc.gov", None)
results = client.get("b7pe-5nws", limit=350000)
moderna_df = pd.DataFrame.from_records(results)
moderna_df["_1st_dose_allocations"] = moderna_df["_1st_dose_allocations"].astype(int)
moderna_df["_2nd_dose_allocations"] = moderna_df["_2nd_dose_allocations"].astype(int)
moderna_df = moderna_df.groupby(['jurisdiction']).sum()
moderna_df = moderna_df.reset_index()
moderna_df=moderna_df.rename(columns={"_1st_dose_allocations":"Moderna_1st_Dose", "_2nd_dose_allocations":"Moderna_2nd_Dose"})

In [4]:
client = Socrata("data.cdc.gov", None)
results = client.get("saz5-9hgg", limit=350000)
pfizer_df = pd.DataFrame.from_records(results)
pfizer_df["_1st_dose_allocations"] = pfizer_df["_1st_dose_allocations"].astype(int)
pfizer_df["_2nd_dose_allocations"] = pfizer_df["_2nd_dose_allocations"].astype(int)
pfizer_df = pfizer_df.groupby(['jurisdiction']).sum()
pfizer_df = pfizer_df.reset_index()
pfizer_df=pfizer_df.rename(columns={"_1st_dose_allocations":"Pfizer_1st_Dose", "_2nd_dose_allocations":"Pfizer_2nd_Dose"})

In [5]:
client = Socrata("data.cdc.gov", None)
results = client.get("w9zu-fywh", limit=350000)
jnj_df = pd.DataFrame.from_records(results)
jnj_df["_1st_dose_allocations"] = jnj_df["_1st_dose_allocations"].astype(int)
jnj_df = jnj_df.groupby(['jurisdiction']).sum()
jnj_df = jnj_df.reset_index()
jnj_df=jnj_df.rename(columns={"_1st_dose_allocations":"JnJ_1st_Dose"})

In [6]:
vaccine_df=pfizer_df.merge(moderna_df, how="outer", on="jurisdiction")
vaccine_df=vaccine_df.merge(jnj_df,how="outer", on="jurisdiction")
vaccine_df=vaccine_df.reset_index()
vaccine_df=vaccine_df.rename(columns={"jurisdiction":"state"})
vaccine_df= vaccine_df.drop(columns=["index"])

In [7]:
states_df = pd.read_csv("us-states-territories.csv", encoding="unicode_escape")
states_df=states_df.drop(columns=["Capital","Population (2015)"])
# states_df = states_df.loc[states_df["Abbreviation"] == "NaN"]
states_df=states_df.rename(columns={"Name":"state"})

In [8]:
vaccine_df_by_state = states_df.merge(vaccine_df, on="state", how="inner")
vaccine_df_by_state = vaccine_df_by_state.drop(columns=["Unnamed: 7", "Type"])

In [9]:
final_data = vaccine_df_by_state.merge(doom_df, on="state", how="inner")

In [10]:
client = Socrata("data.cdc.gov", None)
results = client.get("djj9-kh3p", limit=350000)
hesitency_df = pd.DataFrame.from_records(results)
hesitency_df = hesitency_df[["state","estimated_hesitant","estimated_unsure_or_hesitant","estimated_strongly_hesitant"]]
hesitency_df["estimated_hesitant"] = hesitency_df["estimated_hesitant"].astype(float)
hesitency_df["estimated_unsure_or_hesitant"] = hesitency_df["estimated_unsure_or_hesitant"].astype(float)
hesitency_df["estimated_strongly_hesitant"] = hesitency_df["estimated_strongly_hesitant"].astype(float)
hesitency_df = hesitency_df.groupby(["state"])["estimated_hesitant"].agg(mean_hesitency="mean",max_hesitency="max",min_hesitency="min")
hesitency_df = hesitency_df.reset_index()

In [11]:
final_data_df = final_data.merge(hesitency_df, on="state", how="left")
final_data_df = final_data_df.fillna(0)
final_data_df["population_2019"] =final_data_df["population_2019"].astype(int)
final_data_df["area"] = final_data_df["area"].astype(int)
final_data_df["density_per_sq_mile"] = round(final_data_df["population_2019"]/final_data_df["area"], 2)


In [12]:
final_data_df = final_data_df.set_index("state")
final_data_df.head()

,Abbreviation,population_2019,area,Pfizer_1st_Dose,Pfizer_2nd_Dose,Moderna_1st_Dose,Moderna_2nd_Dose,JnJ_1st_Dose,covid_19_deaths,mean_hesitency,max_hesitency,min_hesitency,density_per_sq_mile
state,,,,,,,,,,,,,
Alabama,AL,4903185,52420,1552740,1552740,1295260,1295260,179400,12225,16.485588,18.98,12.44,93.54
Alaska,AK,731545,665384,369960,358260,285860,285860,39900,409,18.826000,23.61,15.24,1.10
Arizona,AZ,7278717,113990,2203260,2203260,1835960,1835960,253900,16957,14.272037,21.05,9.31,63.85
Arkansas,AR,3017804,53179,951600,951600,786220,786220,109000,7298,20.626500,22.79,17.15,56.75
California,CA,39512223,163695,12333360,12333360,10329700,10329700,1428400,68116,6.166000,9.76,2.76,241.38


In [13]:
client = Socrata("data.cdc.gov", None)
results = client.get("8xkx-amqh", limit=1000000)
adminstration_df = pd.DataFrame.from_records(results)
adminstration_df = adminstration_df.fillna(0)
admin_data = adminstration_df[["recip_county","recip_state","series_complete_pop_pct","series_complete_yes","administered_dose1_recip_65plus","administered_dose1_recip_65pluspop_pct","svi_ctgy"]]
admin_data = admin_data.rename(columns={"recip_state":"Abbreviation","series_complete_pop_pct":"pct_pop_vax","series_complete_yes":"total_pop_vax","administered_dose1_recip_65plus":"age_65_plus_pop_vaxed","administered_dose1_recip_65pluspop_pct":"pct_65_plus_vaxed","svi_ctgy":"Social_Vulnerability_Index"})
admin_data["pct_pop_vax"] = admin_data["pct_pop_vax"].astype(float)
admin_data["total_pop_vax"] = admin_data["total_pop_vax"].astype(int)
admin_data["age_65_plus_pop_vaxed"] = admin_data["age_65_plus_pop_vaxed"].astype(int)
admin_data["pct_65_plus_vaxed"] = admin_data["pct_65_plus_vaxed"].astype(float)
admin_data = admin_data.groupby(["Abbreviation","recip_county"]).max()
admin_data = admin_data.reset_index()
admin_data

,Abbreviation,recip_county,pct_pop_vax,total_pop_vax,age_65_plus_pop_vaxed,pct_65_plus_vaxed,Social_Vulnerability_Index
0,AK,Aleutians East Borough,67.7,2260,166,47.3,Mod-High
1,AK,Aleutians West Census Area,52.9,2983,273,65.2,Low-Mod
2,AK,Anchorage Municipality,50.2,144637,31020,91.9,Low-Mod
3,AK,Bethel Census Area,54.4,10011,1251,86.4,High
4,AK,Bristol Bay Borough,87.7,733,123,90.4,Low-Mod
...,...,...,...,...,...,...,...
3277,WY,Teton County,77.5,18184,3483,93.7,Low
3278,WY,Uinta County,36.5,7379,2535,83.8,Low-Mod
3279,WY,Unknown County,0.0,8448,4208,0.0,0
3280,WY,Washakie County,33.3,2600,1319,76.2,Low-Mod


In [14]:
# aggregating to state-level data
# all mins are zero, so just skipping that value from the aggregation
vaxed_by_state = admin_data.groupby("Abbreviation").agg(
        avg_pct_vax=pd.NamedAgg(column='pct_pop_vax', aggfunc='mean'),
        max_pct_vax=pd.NamedAgg(column='pct_pop_vax', aggfunc=max),
        total_pop_vax=pd.NamedAgg(column='total_pop_vax', aggfunc=sum),
        age_65_plus_vax=pd.NamedAgg(column='age_65_plus_pop_vaxed', aggfunc=sum),
        avg_pct_65up_vax=pd.NamedAgg(column='pct_65_plus_vaxed', aggfunc='mean'),
        max_pct_65up_vax=pd.NamedAgg(column='pct_65_plus_vaxed', aggfunc=max)
)
vaxed_by_state = vaxed_by_state.reset_index()
vaxed_by_state.head()

,Abbreviation,avg_pct_vax,max_pct_vax,total_pop_vax,age_65_plus_vax,avg_pct_65up_vax,max_pct_65up_vax
0,AK,51.686667,87.7,352639,81161,76.830000,97.6
1,AL,33.430882,50.2,1956859,726229,76.041176,100.0
2,AR,35.986842,48.2,1307990,442466,70.072368,91.2
3,AS,0.000000,0.0,26666,2821,0.000000,0.0
4,AZ,48.450000,86.3,3600809,1153123,78.687500,100.0


In [15]:
# created new column for merging because couldn't figure out why 'Abbreviation' column wouldn't merge correctly, this worked
vaxed_by_state['state'] = ['Alaska', 'Alabama', 'Arkansas', 'American Samoa', 'Arizona', 'California', 'Colorado', 'Connecticut', 
    'District of Columbia', 'Delaware', 'Florida', 'FM', 'Georgia', 'Guam', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana',
    'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'MH', 'Michigan', 'Minnesota', 'Missouri', 'MP',
    'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada',
    'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'PW', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Unknown', 'Utah', 'Virginia', 'Virgin Islands', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']

In [16]:
vaxed_by_state.head()

,Abbreviation,avg_pct_vax,max_pct_vax,total_pop_vax,age_65_plus_vax,avg_pct_65up_vax,max_pct_65up_vax,state
0,AK,51.686667,87.7,352639,81161,76.830000,97.6,Alaska
1,AL,33.430882,50.2,1956859,726229,76.041176,100.0,Alabama
2,AR,35.986842,48.2,1307990,442466,70.072368,91.2,Arkansas
3,AS,0.000000,0.0,26666,2821,0.000000,0.0,American Samoa
4,AZ,48.450000,86.3,3600809,1153123,78.687500,100.0,Arizona


In [17]:
final_data_2 = final_data_df.merge(vaxed_by_state, on="state", how="inner")
# dropping 2nd Abbreviation column
final_data_2 = final_data_2.drop(columns='Abbreviation_y')
# rename Abbreviation_x
final_data_2 = final_data_2.rename(columns={'Abbreviation_x': 'Abbreviation'})
final_data_2['unvaxed_pop'] = final_data_2['population_2019'] - final_data_2['total_pop_vax']
final_data_2 = final_data_2.set_index("state")

In [18]:
# comparing previous final dataframe to final_data_2
list(final_data_df.columns.values)

['Abbreviation',
 'population_2019',
 'area',
 'Pfizer_1st_Dose',
 'Pfizer_2nd_Dose',
 'Moderna_1st_Dose',
 'Moderna_2nd_Dose',
 'JnJ_1st_Dose',
 'covid_19_deaths',
 'mean_hesitency',
 'max_hesitency',
 'min_hesitency',
 'density_per_sq_mile']

In [19]:
# comparing previous final dataframe to final_data_2
list(final_data_2.columns.values)

['Abbreviation',
 'population_2019',
 'area',
 'Pfizer_1st_Dose',
 'Pfizer_2nd_Dose',
 'Moderna_1st_Dose',
 'Moderna_2nd_Dose',
 'JnJ_1st_Dose',
 'covid_19_deaths',
 'mean_hesitency',
 'max_hesitency',
 'min_hesitency',
 'density_per_sq_mile',
 'avg_pct_vax',
 'max_pct_vax',
 'total_pop_vax',
 'age_65_plus_vax',
 'avg_pct_65up_vax',
 'max_pct_65up_vax',
 'unvaxed_pop']

In [20]:
# renaming final_data_2 to final_data_df because the merges didn't lose anything
final_data_df = final_data_2

In [21]:
final_data_df

,Abbreviation,population_2019,area,Pfizer_1st_Dose,Pfizer_2nd_Dose,Moderna_1st_Dose,Moderna_2nd_Dose,JnJ_1st_Dose,covid_19_deaths,mean_hesitency,max_hesitency,min_hesitency,density_per_sq_mile,avg_pct_vax,max_pct_vax,total_pop_vax,age_65_plus_vax,avg_pct_65up_vax,max_pct_65up_vax,unvaxed_pop
state,,,,,,,,,,,,,,,,,,,,
Alabama,AL,4903185,52420,1552740,1552740,1295260,1295260,179400,12225,16.485588,18.98,12.44,93.54,33.430882,50.2,1956859,726229,76.041176,100.0,2946326
Alaska,AK,731545,665384,369960,358260,285860,285860,39900,409,18.826000,23.61,15.24,1.10,51.686667,87.7,352639,81161,76.830000,97.6,378906
Arizona,AZ,7278717,113990,2203260,2203260,1835960,1835960,253900,16957,14.272037,21.05,9.31,63.85,48.450000,86.3,3600809,1153123,78.687500,100.0,3677908
Arkansas,AR,3017804,53179,951600,951600,786220,786220,109000,7298,20.626500,22.79,17.15,56.75,35.986842,48.2,1307990,442466,70.072368,91.2,1709814
California,CA,39512223,163695,12333360,12333360,10329700,10329700,1428400,68116,6.166000,9.76,2.76,241.38,43.137288,75.9,22475646,5738386,75.498305,100.0,17036577
Colorado,CO,5758736,104094,1765065,1765065,1468800,1468800,203400,7504,7.132619,9.58,4.94,55.32,46.526154,84.5,3975182,1235697,70.321538,98.5,1783554
Connecticut,CT,3565278,5543,1177230,1177230,972620,972620,134800,8422,5.516154,7.35,3.76,643.20,57.466667,73.0,2384751,621579,86.088889,100.0,1180527
Delaware,DE,973764,2489,326820,326820,257840,257840,35700,1827,5.793333,6.64,4.56,391.23,39.200000,56.5,547024,182592,68.675000,96.9,426740
Florida,FL,21477737,65758,6745860,6745860,5650460,5650460,781500,44958,12.574702,17.95,8.33,326.62,43.438235,71.8,11736562,4281373,85.901471,100.0,9741175


In [23]:
final_data_df["Total_Allocation"] = final_data_df["Pfizer_1st_Dose"] + final_data_df["Moderna_1st_Dose"] + final_data_df["JnJ_1st_Dose"]

In [25]:
final_data_df

,Abbreviation,population_2019,area,Pfizer_1st_Dose,Pfizer_2nd_Dose,Moderna_1st_Dose,Moderna_2nd_Dose,JnJ_1st_Dose,covid_19_deaths,mean_hesitency,...,min_hesitency,density_per_sq_mile,avg_pct_vax,max_pct_vax,total_pop_vax,age_65_plus_vax,avg_pct_65up_vax,max_pct_65up_vax,unvaxed_pop,Total_Allocation
state,,,,,,,,,,,,,,,,,,,,,
Alabama,AL,4903185,52420,1552740,1552740,1295260,1295260,179400,12225,16.485588,...,12.44,93.54,33.430882,50.2,1956859,726229,76.041176,100.0,2946326,3027400
Alaska,AK,731545,665384,369960,358260,285860,285860,39900,409,18.826000,...,15.24,1.10,51.686667,87.7,352639,81161,76.830000,97.6,378906,695720
Arizona,AZ,7278717,113990,2203260,2203260,1835960,1835960,253900,16957,14.272037,...,9.31,63.85,48.450000,86.3,3600809,1153123,78.687500,100.0,3677908,4293120
Arkansas,AR,3017804,53179,951600,951600,786220,786220,109000,7298,20.626500,...,17.15,56.75,35.986842,48.2,1307990,442466,70.072368,91.2,1709814,1846820
California,CA,39512223,163695,12333360,12333360,10329700,10329700,1428400,68116,6.166000,...,2.76,241.38,43.137288,75.9,22475646,5738386,75.498305,100.0,17036577,24091460
Colorado,CO,5758736,104094,1765065,1765065,1468800,1468800,203400,7504,7.132619,...,4.94,55.32,46.526154,84.5,3975182,1235697,70.321538,98.5,1783554,3437265
Connecticut,CT,3565278,5543,1177230,1177230,972620,972620,134800,8422,5.516154,...,3.76,643.20,57.466667,73.0,2384751,621579,86.088889,100.0,1180527,2284650
Delaware,DE,973764,2489,326820,326820,257840,257840,35700,1827,5.793333,...,4.56,391.23,39.200000,56.5,547024,182592,68.675000,96.9,426740,620360
Florida,FL,21477737,65758,6745860,6745860,5650460,5650460,781500,44958,12.574702,...,8.33,326.62,43.438235,71.8,11736562,4281373,85.901471,100.0,9741175,13177820


In [24]:
admin_data.to_json("administration.json")
final_data_df.to_json("final_data.json")